In [1]:
import os
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#import translated dataframe
data = pd.read_pickle("/home/yosr/Downloads/TraintopicsTranslated.pkl")
data

data_text = pd.concat([data['translated_text'],data['categories']],axis=1)
data_text.head()

,translated_text,categories
0,GIVE ME A NIGHT by LOS INQUIETOS (BEsAs) - JON...,Arts & Entertainment
1,YouTube Doki - Doki Gets BulliedDoki tries to ...,Arts & Entertainment
2,Concrete Floor Leveling Tool a Paper Roller: W...,Business & Industrial
3,Mi-T-M 3500psi 4 gpm Pressure Washer Demonstra...,"Autos & Vehicles,Home & Garden,Business & Indu..."
4,Pes 15 Barclays patch and kitspatch Barclays P...,"Games,Sports"


In [6]:
#for this algorithm we will use them as lists
#turn categories to lists

list= []
i = 0
for ch in data_text['categories']:
    list = ch.split(",")
    data_text.at[i, 'categories'] = list
    i += 1
    

In [8]:
data_text['categories'].values.tolist()
data_text

,translated_text,categories
0,GIVE ME A NIGHT by LOS INQUIETOS (BEsAs) - JON...,[Arts & Entertainment]
1,YouTube Doki - Doki Gets BulliedDoki tries to ...,[Arts & Entertainment]
2,Concrete Floor Leveling Tool a Paper Roller: W...,[Business & Industrial]
3,Mi-T-M 3500psi 4 gpm Pressure Washer Demonstra...,"[Autos & Vehicles, Home & Garden, Business & I..."
4,Pes 15 Barclays patch and kitspatch Barclays P...,"[Games, Sports]"
5,CCL2 Final Karnataka Bulldozers Vs Chennai Rhi...,"[Games, Sports]"
6,iCaro B4 4ch Channel Sony CCD IR Cameras DVR G...,"[Internet & Telecom, Computers & Electronics]"
7,Destiny: Hunt for Shaders Ep. 7 DEAD ORBIT HYP...,[Games]
8,Sonic the Hedgehog (06) playthrough [Shadow] [...,"[Games, Arts & Entertainment]"
9,"Ultrasonic washing for fruits, vegetables, ber...",[Food & Drink]


In [9]:
def cleanPunc(sentence): #function to clean the word of any punctuation or special characters
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    cleaned = cleaned.strip()
    cleaned = cleaned.replace("\n"," ")
    return cleaned

#function to remove digits just to make sure
def removeCharDigit(text):
    str='`1234567890-=~@#$%^&*()►■●•_+[!{;":\'><.,/?"}]'
    for w in text:
        if w in str:
            text=text.replace(w,'')
    return text

def keepAlpha(sentence):
    alpha_sent = ""
    for word in sentence.split():
        alpha_word = re.sub('[^a-z A-Z]+', ' ', word)
        alpha_sent += alpha_word
        alpha_sent += " "
    alpha_sent = alpha_sent.strip()
    return alpha_sent

#function to remove accented chars
import unicodedata
def removeAscendingChar(data):
    data=unicodedata.normalize('NFKD', data).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return data

In [12]:
#apply changes
import re
#put all the unctions above into one global function
def PreProcessing(text):
    text=removeAscendingChar(text)
    #print(text)
    text=cleanPunc(text)
    #print(text)
    text=keepAlpha(text)
    #print(text)
    text=removeCharDigit(text)
    return(text)


data_text['translated_text'] = data_text['translated_text'].str.lower()
#apply the function preprocess to our data
data_text['translated_text'] = data_text['translated_text'].map(lambda com : PreProcessing(com))

data_text.head()

,translated_text,categories
0,give me a night by los inquietos besas jonna...,[Arts & Entertainment]
1,youtube doki doki gets bullieddoki tries to ...,[Arts & Entertainment]
2,concrete floor leveling tool a paper roller w...,[Business & Industrial]
3,mi t m psi gpm pressure washer demonstratio...,"[Autos & Vehicles, Home & Garden, Business & I..."
4,pes barclays patch and kitspatch barclays pr...,"[Games, Sports]"


In [13]:
#removing stopwords
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
stop_words.update(['zero','one','two','three','four','five','six','seven','eight','nine','ten','may','also','across','among','beside','however',
                   'yet','within','www','e','de','la','el','v','www','http','com','https','a','b','c',
                  'd','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','z','x','youtub'])
re_stop_words = re.compile(r"\b(" + "|".join(stop_words) + ")\\W", re.I)

def removeStopWords(sentence):
    global re_stop_words
    return re_stop_words.sub(" ", sentence)

data_text['translated_text'] = data_text['translated_text'].apply(removeStopWords)
data_text.head()

,translated_text,categories
0,give night los inquietos besas jonnatangc...,[Arts & Entertainment]
1,youtube doki doki gets bullieddoki tries re...,[Arts & Entertainment]
2,concrete floor leveling tool paper roller ...,[Business & Industrial]
3,mi psi gpm pressure washer demonstrationt...,"[Autos & Vehicles, Home & Garden, Business & I..."
4,pes barclays patch kitspatch barclays premi...,"[Games, Sports]"


In [14]:
#stemming
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")
def stemming(sentence):
    stemSentence = ""
    for word in sentence.split():
        stem = stemmer.stem(word)
        stemSentence += stem
        stemSentence += " "
    stemSentence = stemSentence.strip()
    return stemSentence

data_text['translated_text'] = data_text['translated_text'].apply(stemming)
#df.head()

In [15]:
#tfidf
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=5000)
x = tfidf_vectorizer.fit_transform(data_text['translated_text'])


In [16]:
#transforming text to features

from sklearn.preprocessing import MultiLabelBinarizer

multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(data_text['categories'])



# transform target variable
y = multilabel_binarizer.transform(data_text['categories'])
y

array([[0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 1, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0]])

In [17]:
from sklearn.model_selection import train_test_split

xtrain, xtest, ytrain, ytest = train_test_split(x, y, random_state=42, test_size=0.30, shuffle=True)

print(xtrain.shape)
print(xtest.shape)

(12451, 5000)
(5337, 5000)


In [18]:
 categories = ['Arts & Entertainment', 'Sports', 'Food & Drink', 'Jobs & Education', 'News', 'Health', 'Real Estate', 'Pets & Animals', 'Home & Garden', 'People & Society', 'Travel', 'Beauty & Fitness', 'Autos & Vehicles', 'Finance', 'Shopping', 'Internet & Telecom', 'Law & Government', 'Business & Industrial', 'Reference', '(Unknown)', 'Computers & Electronics', 'Hobbies & Leisure', 'Games', 'Books & Literature', 'Science']

In [57]:
# using binary relevance
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
# initialize binary relevance multi-label classifier
# with a gaussian naive bayes base classifier
classifier = BinaryRelevance(GaussianNB())

# train
classifier.fit(xtrain, ytrain)

# predict
predictions = classifier.predict(xtest)

# accuracy
print("Accuracy = ",accuracy_score(ytest,predictions))
print("\n")

Accuracy =  0.10249203672475174




In [59]:
import sklearn.metrics as metrics

metrics.hamming_loss(ytest, predictions)

0.18677908937605397

In [58]:
classifier.classifiers_

[GaussianNB(priors=None, var_smoothing=1e-09),
 GaussianNB(priors=None, var_smoothing=1e-09),
 GaussianNB(priors=None, var_smoothing=1e-09),
 GaussianNB(priors=None, var_smoothing=1e-09),
 GaussianNB(priors=None, var_smoothing=1e-09),
 GaussianNB(priors=None, var_smoothing=1e-09),
 GaussianNB(priors=None, var_smoothing=1e-09),
 GaussianNB(priors=None, var_smoothing=1e-09),
 GaussianNB(priors=None, var_smoothing=1e-09),
 GaussianNB(priors=None, var_smoothing=1e-09),
 GaussianNB(priors=None, var_smoothing=1e-09),
 GaussianNB(priors=None, var_smoothing=1e-09),
 GaussianNB(priors=None, var_smoothing=1e-09),
 GaussianNB(priors=None, var_smoothing=1e-09),
 GaussianNB(priors=None, var_smoothing=1e-09),
 GaussianNB(priors=None, var_smoothing=1e-09),
 GaussianNB(priors=None, var_smoothing=1e-09),
 GaussianNB(priors=None, var_smoothing=1e-09),
 GaussianNB(priors=None, var_smoothing=1e-09),
 GaussianNB(priors=None, var_smoothing=1e-09),
 GaussianNB(priors=None, var_smoothing=1e-09),
 GaussianNB(p

In [60]:
from sklearn.metrics import f1_score
f1_score(ytest, predictions, average="micro")

0.3055315591472761

In [61]:
#imporove f1 score
# predict probabilities
y_pred_prob = clf.predict_proba(xtest)

#setting a new threshold value
t = 0.3 # threshold value
y_pred_new = (y_pred_prob >= t).astype(int)

# evaluate performance
f1_score(ytest, y_pred_new, average="micro")

0.5976722655733369

In [62]:
# accuracy
print("Accuracy = ",accuracy_score(ytest,y_pred_new))
print("\n")

Accuracy =  0.3711823121603897




In [63]:
metrics.hamming_loss(ytest, y_pred_new)

0.045598650927487355

In [45]:
# using classifier chains
from skmultilearn.problem_transform import ClassifierChain
from sklearn.linear_model import LogisticRegression


# initialize classifier chains multi-label classifier
classifier2 = ClassifierChain(LogisticRegression())

# Training logistic regression model on train data
classifier2.fit(xtrain, ytrain)

# predict
predictions2 = classifier.predict(xtest)

# accuracy
print("Accuracy = ",accuracy_score(ytest,predictions2))
print("\n")

/home/yosr/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/yosr/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/yosr/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/yosr/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/yosr/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solv

Accuracy =  0.3893573168446693




In [64]:
metrics.hamming_loss(ytest, predictions2)

0.05454000374742365

In [47]:
from sklearn.metrics import f1_score
f1_score(ytest, predictions2, average="micro")

#imporove f1 score
# predict probabilities
#y_pred_prob = clf.predict_proba(xtest)

#setting a new threshold value
#t = 0.3 # threshold value
#y_pred_new = (y_pred_prob >= t).astype(int)

# evaluate performance
#f1_score(ytest, y_pred_new, average="micro")

# accuracy
#print("Accuracy = ",accuracy_score(ytest,y_pred_new))
#print("\n")

0.5101312689330192

In [48]:
#imporove f1 score
# predict probabilities
y_pred_prob = classifier2.predict_proba(xtest)

#setting a new threshold value
t = 0.3 # threshold value
y_pred_new = (y_pred_prob >= t).astype(int)

# evaluate performance
f1_score(ytest, y_pred_new, average="micro")


0.5725873659647759

In [49]:
# accuracy
print("Accuracy = ",accuracy_score(ytest,y_pred_new))
print("\n")

Accuracy =  0.31534569983136596




In [65]:
# using Label Powerset
from skmultilearn.problem_transform import LabelPowerset


# initialize label powerset multi-label classifier
classifier = LabelPowerset(LogisticRegression())

# train
classifier.fit(xtrain, ytrain)

# predict
predictions3 = classifier.predict(xtest)

# accuracy
print("Accuracy = ",accuracy_score(ytest,predictions3))
print("\n")

/home/yosr/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/yosr/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Accuracy =  0.3893573168446693




In [66]:
metrics.hamming_loss(ytest, predictions3)

0.05454000374742365

In [53]:
from sklearn.metrics import f1_score
f1_score(ytest, predictions3, average="micro")

0.5101312689330192

In [55]:
#imporove f1 score
# predict probabilities
y_pred_prob = classifier.predict_proba(xtest)

#setting a new threshold value
t = 0.3 # threshold value
y_pred_new = (y_pred_prob >= t).astype(int)

# evaluate performance
f1_score(ytest, y_pred_new, average="micro")

0.4566244854398537

In [56]:
# accuracy
print("Accuracy = ",accuracy_score(ytest,y_pred_new))
print("\n")

Accuracy =  0.31534569983136596




In [32]:
train, test = train_test_split(data_text, random_state=42, test_size=0.20, shuffle=True)

In [33]:
#tfidf
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=5000)
x_train = tfidf_vectorizer.fit_transform(train['translated_text'])
x_test = tfidf_vectorizer.fit_transform(test['translated_text'])

In [34]:
#transforming text to features

from sklearn.preprocessing import MultiLabelBinarizer

multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(train['categories'])
multilabel_binarizer.fit(test['categories'])



# transform target variable
y_train = multilabel_binarizer.transform(train['categories'])
y_test = multilabel_binarizer.transform(test['categories'])
y_train

array([[0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 1, ..., 0, 0, 1]])

In [41]:
#One vs all 
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier


# Using pipeline for applying logistic regression and one vs rest classifier
LogReg_pipeline = Pipeline([
                ('clf', OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=-1)),
            ])

for category in categories:
    print('**Processing {} videos...**'.format(category))
    
    # Training logistic regression model on train data
    LogReg_pipeline.fit(x_train, y_train)
    
    # calculating test accuracy
    prediction = LogReg_pipeline.predict(x_test)
    print('Test accuracy is {}'.format(accuracy_score(y_test, prediction)))
    print("\n")

**Processing Arts & Entertainment videos...**
Test accuracy is 0.01517706576728499


**Processing Sports videos...**
Test accuracy is 0.01517706576728499


**Processing Food & Drink videos...**
Test accuracy is 0.01517706576728499


**Processing Jobs & Education videos...**
Test accuracy is 0.01517706576728499


**Processing News videos...**
Test accuracy is 0.01517706576728499


**Processing Health videos...**
Test accuracy is 0.01517706576728499


**Processing Real Estate videos...**
Test accuracy is 0.01517706576728499


**Processing Pets & Animals videos...**
Test accuracy is 0.01517706576728499


**Processing Home & Garden videos...**
Test accuracy is 0.01517706576728499


**Processing People & Society videos...**
Test accuracy is 0.01517706576728499


**Processing Travel videos...**
Test accuracy is 0.01517706576728499


**Processing Beauty & Fitness videos...**
Test accuracy is 0.01517706576728499


**Processing Autos & Vehicles videos...**
Test accuracy is 0.01517706576728499

